# Spark

## Spark Introduction

#### What is apache spark

* Its a open-source, `distributed computing system` designed for fast and general-purpose data processing on large-scale datasets.
* OR
* Apache spark is a `unified` computing engine and set of libraries for parallel data processing on computer cluster.

#### Why Apache Spark

* Previously we used to have Database servers which stores structure data.
* The DB where not capable of handlig semi structured data. and same time we received large amount of data
* we started facing issue of Big Data
    * Big Data are 3'Vs (
        * Velocity
        * Variety
        * Volume
    )
    * When we get the large `volume` of data at high `velocity` with different `variety` of datasets then it is termed as `Big Data`

* We got the problem of storing and processing this big data.
* Here we have 2 options to solve this problem:
    * `Monolithic approach`: (scale the system vertically, increase the size of our system by adding more hardware)
    * `distribted Approach`: Horizontal scaling ( add several different parallel systems ) (feature: High availability. Economical, Horizontal Scaling)
* to implement distributed apporach we got `Hadoop` followed by `Spark`.

#### Spark Vs Hadoop

| Parameters| Hadoop | Spark |
|---|---|---|
| performance | Hadoop slow then spark, because it writes data to disk and read it back again from disk to in-memory | Spark is faster then hadoop as it does all computation in memory.|
| file processing | Build for batch data processing | Batch as well as streming data processing. |
|ease of use | Difficult to write code in hadoop, hive was build to make it easy. | Easy to write and debug code. Spark provides low and high level APIs |
| security | Uses Kerberos authentication present within YARN Resource manager and ACL authentication to enter into any netowork. it uses tokens to get the permission. | Dosn't have solid security feature. It uses HDFC for accessing data and uses YARN for resource negotiatin |
|Fault tollerent | It creates replica of data parts in different partition. | Spark creates DAG. if in case RDD is lost, it will regenrate data from DAG, therefore its a bit time consuming|

## Spark Ecosystem

![alt text](images/spark_ecosys.png)

## Spark Architecture

>> Blog: https://www.encora.com/insights/apache-spark-architecture



![alt text](images/spark__architecture.png)

* Apache Spark has a master-slave architecture that consists of a **Driver**, **Executors**, and a **Cluster Manager**.

**Driver Program:**
- The **Driver** is the central coordinator in a Spark application. It is responsible for:
    - Defining the main logic of the Spark application.
    - Creating the **SparkContext**, which serves as the entry point for Spark functionality.
    - Converting transformations into a logical Directed Acyclic Graph (DAG).
    - Submitting jobs to the cluster manager and distributing tasks among the executors.

**Executors:**
- Executors are worker nodes in the cluster responsible for:
    - Executing tasks assigned by the driver.
    - Storing data for the application in memory or disk storage.
    - Sending results back to the driver.

**Cluster Manager:**
- The Cluster Manager oversees resource management and job scheduling. Spark can run on various cluster managers, such as:
    - **Standalone Cluster Manager:** A simple built-in cluster manager.
    - **Apache Mesos:** A general cluster manager that can run Hadoop MapReduce and other applications.
    - **Hadoop YARN:** The resource management layer of Hadoop.
    - **Kubernetes:** An open-source platform for automating deployment, scaling, and operations of application containers.


1. **Job Submission:**
 - The user submits a Spark application using the SparkContext in the driver program.

2. **DAG Construction:**
 - The driver builds a logical Directed Acyclic Graph (DAG) of stages representing transformations and actions.

3. **Task Scheduling:**
 - The DAG is divided into smaller sets of tasks, which are then submitted to the cluster manager.

4. **Task Execution:**
 - The cluster manager allocates resources and schedules the tasks on available executors.
 - Executors perform the tasks assigned to them, which may involve reading data from a data source, performing computations, and storing intermediate results.

5. **Result Collection:**
 - Executors send the results of their computations back to the driver.
 - The driver program consolidates these results and performs any final actions required.


In Spark, operations on data are categorized into **transformations** and **actions**:

- **Transformations:**
 - These are operations that create a new RDD from an existing one. They are lazy, meaning they don’t execute immediately but instead create a logical plan of execution.
 - Eg. `map()`, `filter()`

- **Actions:**
 - These operations trigger the execution of the transformations and return a result to the driver program or write data to an external storage system.
 - Eg.`collect()`, `count()`

## Transformations and Actions

#### What is Transformation and How many type of transformations do we have?

### Transformatinos

* transformations are operations that are applied to Resilient Distributed Datasets (RDDs) to create new RDDs. Transformations are lazy, meaning they do not immediately compute their results but instead, build up a series of transformations to be applied when an action is triggered. This allows Spark to optimize the execution plan and efficiently manage resources.

* There are 2 types of Transformations `Narrow` and `Wide`

##### Narrow Transformation:
* Transformatin that does not require data movement between partitions.
* Transformations which are not dependent on other partition of the data. and which executes with only one partition.

* Eg:
    * filter()
    * map()
    * union()
    * coalesce()
    * select()

##### Wide
* transformations that rquires data movement between paritions.
* Multiple partitions of the data is used to perfrom the operation.
* Eg:
    * repartition()
    * groupByKey()
    * reduceByKey()
    * join()
    * distinct()

### Actions
* actions triggeres the execution of the transofrmations.
* each action creates a new job.
* actios typically returns the result to driver program or writes the data to storage.
* Eg:
    * collect()
    * count()
    * first()
    * save()
    * foreach()
    * reduct()
    * read()
    * mean()
    * inferschema()

#### What happens when we use group by or join in transformation??

* groupBy() and join() typically involves shuffling of data, which makes it a wide transformation.
* groupByKey groups all the values for each key in the RDD. This operation involves the following steps:
    * Partition the data based on key. i.e. all vlaues associated with the same key are moved to same partition.
    * shuffle: data is moved across the partitions.
    * grouping: Spark groups the values by their keys within each partition.
* same goes with join() operation.

#### How Jobs are created in spark??

* for every action a new job is created.
* if there are multiple actions triggered. there will be multiple job createed.

## DAGs and Lazy evaluation

* when the trainsformations are written they are lazily evaluated. i.e. the code will not execure untile the action is triggered.
* till then spark created `DAG` that will have the logicla plan for the exection of the transformations.
* Dag will be created for each job when an action is hit.


## Spark SQL Engine/ Catalyst Optimizer

>> Code -> Catalyst Optimizer -> Java Byte code


* Process involved in Catalyst Optimizer
    >> Code -> Unresolved Logical Plan -> Analysis (Catalog) -> Resolved logical Plan -> Logical Optimization ->Catalyst optimizer ->  Optimized Logical Plan -> Physical Plan -> Best Physical Plan -> Final Code (Set of RDDs to run on different executors.)

### what is catalyst optimizer | spark SQL engine?

* Its a query optimization framework
* Its designed to automatically optimize the execution of SQL query
* It converts the program to java byte code.
* There are 2 types of optimization technique used by catalyst optimzer:
    * Rule Based Optimizer:
        * when submiting a peace of code, execution engin will look at predefined plans, which are used to design logical plan.
        * It will bliendly follow the rules.
    * Cost Based Optimizer:
        * Based on available data it estimates the cost and best effective execution plan is decided.

### How many phases are involved in spark SQL engine to convent a code into Java byte code

* There are 4 Phases involved in spark SQL Engine
    1. Analysis:
        * The unresolved logical plan is analysed. 
        * the catelog contain metadata of the data on which the transformation and action is being performed. 
        * catelog checks whether do we have specifed data regarding the tranformation. if not found we get `AnalysisException`
    2. Logical Optimization:
        * It takes the unsolved paln from analysis phase and optimize it logically.
    3. Physical Plan:
        * There can be multiple physical plan for a logical plan.
        * for all the physical plan the best physical plan is choosen.
        * It used the cost based optimizer to choose the best plan.
    4. Code Generation



### What is catalog?


* It stores the metadata about the data for which the transformation and actions are perofrmed.
* It cheks whether proper metadata is being used in code.

### Why do we get Analysis exception error?

* When the catalog does not find the specific column for the data process. it will throw an analysis exception.
* See the below code for example

In [ ]:
# Create data
from pyspark.sql.functions import monotonically_increasing_id, col, lit, floor, rand

persons = ["person_" + str(i) for i in range(1, 16)]

df = spark.createDataFrame(persons, 'string').toDF('persons')

# Add random weights to each person
df = df.withColumn("person_weight", floor(rand() * 50 + 80))  # Weights between 50 and 100

# Add a unique sequence number
df = df.withColumn("sequence_number", monotonically_increasing_id()+1)

# Show the result
df.show()


df.select('person_name').show()

Error  

---------------------------------------------------------------------------  
AnalysisException                         Traceback (most recent call last)  
<ipython-input-5-12ff14ea2eb3> in <cell line: 18>()  
     16   
     17   
---> 18 df.select('person_name').show()  

### What is physical planning / spark plan?

* Logical plans are converted to multiple physical plan.
* physical plan are the exact execution steps, including specifics about data distribution, join algorithms, and more.
* Most efficient physical plan is selected.

### Is spark SQL engine "a compiler"?


* Yes, it convertes the code into java byte code.

## RDD

#### Whart is RDD

* RDD stands for Reselient Destributed Dataset.
* Its a datastructure 
* and also a low level api
* designed to handle distributed data processing


#### Features
* Fault Tolerant: it stores all the DAGs for each RDD, in case if any RDD is lost, new RDD is created by looking at the DAGs.
* Immutable: becuase of immutability feature, we can recover lost RDD
* Partitioned.
* Lazy_evaluated.
* Comple-time type safety


#### Disadvantages:
* No built-in optimization
* more complex

## Spark Session Vs Spark Context

#### spark context: 

* spark context is the entry point to the spark and defined in org.apache.spark package since 1.0 version .
* It is use to create spark RDD.
* if we want to use additional features of spark like sqlcontext, streamingcontext ect, we require to create seperate  context objects.
* spark shell provides 'sc' variable by default for spark Context object.

#### sparkSession:
* spark Session was introduced in version 2.0 and defined in org.apache.spark.sql.SparkSession package
* It's also a entry point to the spark.
* all context object are encapsulated within it.
* spark shell provides 'spark' variable by default for sparkSession object.

## Job, Stages and Task

#### What are Job, stages and task in apache spark??


1. Application:
    * whenever we hit a spark submit command, that created one applicatoin and that runs on the cluster.
    * One application can have multiple Jobs.

2. Jobs:
    * job is created everytime the action is called.
    * job can have multiple stages.

3. Stages:
    * Its the logical plan that being created.
    * collection of stages is called as DAG.
    * it determines the sequential execution of the program.

4. Task:
    * The logical plan are then converted into physical plan.
    * The stages are then converted into different tasks. that will run on executors.


#### How many Job, stages and task will be created from the code snippet?

![alt text](images/code1.png)

* Number of actions given:
    * read()
    * collect()

* THere are 2 actions therefore 2 jobs will be created.

>> * minimum one stage and one task will be created for each job
>> * For each wide transformation new stage will be created.
>> * Whenever there is a join or groupBy, bydefault we will be having 200 partitions.
>> * Spark will create 2 jobs when we use `inferSchema =True` while reading the data.
>>      * First Job: When you call spark.read.csv with inferSchema=True, Spark launches a job to sample the data and infer the schema. This involves tasks like reading the first few rows of the file or sampling a percentage of the file to determine the structure and data types of columns.
>>      * Second Job: After the schema is inferred, Spark needs to read the data again with the inferred schema. This read operation involves another job because Spark now knows the data types and can parse the CSV file accordingly.

* In the above code, we have 2 wide transformation defined `repartition()` and `groupBy()`.
* For action `read()`

| stage1 |
|---|
| read |
|TASK|
| task1


* for action `collect()`


| Stage2 | Stage3 | Stage4 |
|---|---|---|
| repartition | filter -> select | groupby |
| TASK|||
| stage 1 will have 1 task will create 2 partitions | this 2 partitions will be passed to stage2, which will create 2 task | groupby will create 200 partitions |

## Repartition Vs Coalesce

### What is Repartitioning??

* Repatition is used to evenly distribute the data within the partitions.
* Mostly used when we encounter skewed data.
* There can we situations where some executors completes there execution very quickly but some executors take time.
* its because the executor may be processing partition with more value and data is more skewed in that partition.
* Repatition can be used to increase or decrease partitions.
* PROS:
    * Evenly distribute data.
    * helps to resolve skewness
* Cons:
    * Costly operatins as data shuffling is done.
    * MOre use of resources.

### What is Coalesce??

* Coalesce are generally used to reduce the number of partitions.
* It will merge the small partition to make it a big partition.
* No data shuffling is done.
* less costly opoeration.
* Coalesce can only reduce the partition and cannot increase partitions
* PORS:
    * Cost effective solution.
    * fast and cost effective operation as no data shuffling is done.
* Cons:
    * Data is not evenly distributed.


### Which One will you choose and why??

* Depends on the senerio use it.

* Coalesce:
    * Imagine you have processed a large dataset and are about to write the final results to a file system like HDFS or S3. If your DataFrame has a large number of partitions, each partition will be written as a separate file. This can lead to a large number of small files, which can be inefficient for storage and further processing.

* repartition:
    * Scenario: Balancing Data Before a Join Operation

    * Imagine you have two large DataFrames that you need to join on a key. If the data in these DataFrames is skewed (unevenly distributed across partitions), the join operation can become a bottleneck due to imbalanced workloads.

### Repartition Vs Coalesce

| repartition | Coalesce |
|---|---|
| can increase or descrease partition | can only used to decrease partition |
| more cosly operation | less costly operation |
| shuffles the data | does not shuffles the data |
| create equal distribution of data | can create imbalance data |
| used when performing join on large datasets, if used skewed data it can be a bottleneck | can we used, when writing large dataset into a storage location. strong dataset with many number of partion can create number of part files, to reduce it we can use coalesce |

## Spark Join and Join Stragegy

Video link: https://www.youtube.com/watch?v=ZkJowO83MZ0

### How shuffling is done

* when we have data in 2 different executors and when we apply any joni or group by condition, the operation will create 200 partitions bydefault.
* Now once the data is repartitioined, spark will bring same ID records into same executor,
* the above scenerio is for executors that are in same worker node.
* assume the data is in different worker nodes, then the shuffling operation becomes more costly.
* In this way the shuffling is done.
* after shuffling, Join or groupby operation is performed,


### What are join strategies in spark??

* There are different stategies are used to join or groupby the data
* The strategies are mentioned below:
    * Shuffle Sort-Merge Join `(Bydefauly join used by spark)` `(More CPU power is used)` `(time complexity = O(n log n))`
    * Shuffle hash join `(More memory is used)` `(time complexity = O(1))`
    * broadcast hash join `(time complexity = O(N))`
    * Cartition JOin
    * Broadcaset nested loop join `(time complexity = O(n^2))`

1. Shuffle Sort-Merge Join
    * Once the data shuffling is done.
    * the data is sorted and then it will be merged with matching keys
    * This strategy will use more `CPU power` 
    * It has the time complexity of `O (n log n)`
    * bydefault, spark used this join strategy.

2. Shuffle hash join:
    * once shuffling is done, it will create a hash tabel of the `smallest table` based on keys.
    * this hash keys are then compared with other tables keys and maching records are merged.
    * This strategy will require `more memory`.
    * if strategy used and suffecient memory not found, it will throw `MemoryOutOfBound Exception`
    * it has the time complexity of `O(1)`

3. Broadcast hash join
* > Interview questions:

    * >>When do we need broadcast join??
        * to avoide shuffling operation we use broadcast join.
        * bydefault the size of broadcast table `10MB`. 
        * spark considers tables with less then `10MB` size as small tables and it performs a broadcast join.
        * But spark also suggest that to decide the size of broadcast table depending upon the cluster configuration.
        
    * >> How does broadcast join works??
        * the table which needed to be broadcast will be send to driver and driver will broadcast the table to each executors.

    * >> Differenve between Broadcast hash join and Shuffle hash join??
        * Shuffle hash join: 
            * Shuffling is done to execute the join which is costly operation.
        * Broadcast hash join:
            * Boradcast of table is done for join operation, which increases the execution speed.

    * >> How can we get and sest broadcast size of table??
        * TO get: spark.config.get('spark.sql.autoBroadcastJoinThreshold')
        * To set: spark.config.set('spark.sql.autoBroadcastJoinThreshold', 30485760)  (`30485760 bytes` = `30 MB`)
        * if switch off boradcast: spark.config.set('spark.sql.autoBroadcastJoinThreshold', -1)
    * >> When broadcast table is not good  or where it can fail??
        * when we try to broadcast larger table within the network, then there are chances of failure.




5. Boradcase nested loop join
    * sometimes we have situation where we need to do a cross join
    * in such scenerio this strategy is very usefull.
    * It is the most expensive join strategy
    * It has the time complexity of `O(n^2)`

### Why joins are expensive??

* joins are expensive because they involves transfer of data between executors.
* This transfer of data consumes lot of resources and eventually increases expenses.

### Difference between shuffle hash join and shuffle sort-merge join

| Shuffle Sort-Merge JOin | Shuffle hash Join |
|---|---|
| Its the default Join Strategy | Need to set explicitly |
| Sortes the shuffled data and then joins | Creates hash table from the smaller table and find the value in other table |
| more CPU power is used | More memory is required. |
| time complexity: `O (n log n)` | time complexity: `O(1)` |

## Accumulators

* Accumulators are the global variable that recides on the Driver, and can only read by driver.
* Executors can only update this variable, such as `add()` method.
* Executors have `write_only` access, so they cannot view the data.
* it is suitable to debugging for gathering metrixes.
* Use cases: 
    * counting events
    * debugging and Monitoring

* Types of accumulators:
    * Numeric Accumulators: Used for numerical aggregation operations like sum, count, etc.
    * Custom Accumulators: Custom types of accumulators can be created by extending the AccumulatorV2 class.

## Spark Memory Management | Driver Out of Memory exception

https://www.youtube.com/watch?v=2Eb9mE7pfgs

### What is Driver `java.lang.OutOfMemoryError`(OOM) in spark??

* Its the type of Error that raised the memory overhead issue

### Why Do we get Driver Out of Memory (OOM) Exception??

* java.lang.OutOfMemoryError occurs when the collected data size from all executors are greater then the allocated memory in the Driver.
* For Eg: If i have the driver memory of `1GB` and we arr trying to extract data from all executors which in total is more then 1GB, then the error occurs.

>> NOTE: In driver we have 2 types of memory: 
>>* spark.driver.memory: that handles JVM Process
>>* spark.driver.memoryOverhead: that handles non-JVM Process, such as maintating executors info, details about the application container etc. Any type of objects that get created are stored here.
>>* Whenever we assign some amount of memory to Driver, 10% of the memory or 384 MB of memory, whichever is gratest will be assigned to `memoryOverhead`.

## What is Driver Overhead memory??

* Memory that is within driver that handles `non-JVM` operations is called the Driver Overhead Memory
* Whenever we assign some amount of memory to Driver, 10% of the memory or 384 MB of memory, whichever is gratest will be assigned to `memoryOverhead`.
* Eg: if we assigned 1GB of driver memory, so from that 10% or 384 MB will be assigned to Driver memory overhead.
    * Here 10% of 1GB is 100MB which is less then 384 MB, therefore 384 MB of the memory will be assigned to Driver Memory Overhead.

## Common Reason for `java.lang.OutOfMemoryError` ??

* The 3 common reasons for the above error:
    1. Using `collect()` method:
        * we must have used `collect()` method within code. 
        * `collect()` will pull all data from all executors and collect its in driver memory. 
    2. Boradcost join:
        * There must be a `Broadcast` join used. when larger table is `Broadcasted`.
    3. More Object is used in the process (When the Overhead memory is full)

## How to handle `java.lang.OutOfMemoryError` ??

* Check whether we are using collect() anywhere.
* if Broadcast used, make sure the broadcast table is small enough.
* make sure the Overhead memory is large enough to handle all the object and container info.

## Spark Memory management | Exception out of memory

### Executor Structure

![alt text](images/executor%20struct.png)

* Whenever we request a executor with specific memort and cores, we will be having `Overhead Memory` and `Executor Memory`.
1. **Overhead Memory:**
    * It is the 10% extra memory then requested memory.
    * Eg: if we requested 10Gb of memory, spark will get 10% of 10GB = 1GB extra memory to Overhead Memory.
    * It is used to store `container informaton (around 300-400MB)`, `Non-JVM Objects (pyspark application object)`
2. **Executor Memory:**

    * **Reserved Memory:**
        * 300MB fix memory size is assigned to Reserve storage.
        * it is used to store spark internal objects used by spark engen.
        * >> NOTE: we need to create executor of more then 1.5 times of 300MB = 450 MB. if executor memory with less then 450 MB is created, the process will fail.
    * **User Memory:**
        * It is used to store UDF(User define function), User defined data structure or spark internal metadata.
        * It stores RDD operations, like `aggrigate` -> Map partition transformation.
        * Out of remaining, 40% memory is assigned to user storage.
        * Eg: 40% of 9.7 GB = 3880 MB
        * Memory distribution fraction between *User memory* and *Spark memory* can be changed using configuratio: `spark.memory.fraction`
        * minimum 70:30 ration is suggested.
        
    * **Spark Memory:**
        * It stores dataframe operations.
        * Out of remaining, 60% memory is assigned to user storage.
        * Eg: 60% of 9.7 GB = 5820 MB
        * Memory distribution fraction between *User memory* and *Spark memory* can be changed using configuratio: `spark.memory.fraction`
        * minimum 70:30 ration is suggested.
        * It is devide into 2 sections:
            * **Storage Memory Pool:**
                * It is used to store intermediate stages of the task like joins.
                * It stores catched data.
                * Memory evacuation is done in LRU (least Recently used) fashion
                * 50% of the spark memory is assigned here.
                * Eg: 50% of 5820 MB = 2910 MB
            * **Executor Memory Pool:**
                * This memory will remail until the exevution of process. once process is completed, this memory become empty
                * It stores object that is required during process execution.
                * store hash table for hash aggrigation.
                * *Shorte life span* -> cleaned after every execution.
                * spilling in disk
            
            * TO manage the memory of `Storage Memory Pool` and `Executor Memory Pool` we have got `Static Memory Manager` and `Unified Memory Manager`.
            * `Static Memory Manager` 
                * its the old depricated manager used before spark-1.6 version. 
                * In this the storage was statically defined and would not change after defined.
                * Hard margin memory is defined.
            * `Unified Memory Manager` 
                * its used after spark-1.6 version. 
                * In this, `Storage Memory Pool` and `Executor Memory Pool` can use each other memory if the memory is idea.
                * There is no hard margin defined.
                * *Assume*, we have done catching, it will be soted in `Storage Memory Pool`, but if `Storage Memory Pool` get full and `Executor Memory Pool` has some available memory, then it will catch the data in `Executor Memory Pool` as well.
                * Now, if there are some exections needed to be done, they will run on `Executor Memory Pool` and if `Executor Memory Pool` get full, then it starts removing the occupied *catche* data stored in `Executor Memory Pool` on LRU basis (Least Recently used) and use those memory space.

    

### Why do we get Out Of Memory when data can be spilled to the disk??

* assume the `Executor Memory pool` and ` Storage memory pool` is full. now if we performm `join` operation, the data will get shuffled, and the aim of shuffling is to bring similar data in one partition, assume the size of similar data is more then the availabe memory space in `Executor Memory pool`, then the process get stuck in between and it will not spill the data to disk, here we get the choke point and we get OUt Of Memory Error.
* To Resolve this isse, we need to use `repartition` or `salting`.

## How spark manages storage inside executors internally??

* Executor has `Over head memory`and `Executor Memory` memory defined.
* inside `Executor Memory` we have, `Reserved memory`, `spark memory` and `user memory`.
* `spark memory` is further devided into `storage memory pool` and `executor memory pool`.
* in this way executor handles storage

## How task are splitted in executors??

* each task will be run in each core parallelly.
* and all this cores runs in `executor memoy pool`.
* >> NOTE: Its recomended that to give 3-5 core per executors for better performance.

## Why do we need overhead memory in executors??

* it stores container info and non-jvm objects.

## when do we get executor Out of Memoey Exception??

* When there is no extra memory in `Executor memory pool` or `storage memory pool` or `executor Overhead` gets full or `user memory` is full
* There can be multiple reasons.

## Type of memory managers in spark??

* There are 2 main types of memory managers:
    * Static memory manager.
    * Unified memory manager.

## Spark Submit

#### What is Spark Submit

* Spark is a command line tool, which is used to run spark code.
* Spark submit makes the package of all the jar files and code and take it to the cluster to run.

#### How do you run your program on spark Cluster??

* We run a program on cluster using spark submit.
* Spark submit makes a package of code and jar files and send it to cluster.

#### Where is your spark cluster??

* there are different type of cluster. 
    * Stand alone cluster.
    * Local
    * kubernetes (K8)
    * Yarn

#### What is deploy mode in spark submit??

* There are 2 types of deployment modes.
    * Client mode
    * Cluster mode

#### What is master in Spark Submit??

* The `--master` option in spark-submit is used to specify the master URL for the cluster manager that Spark should connect to the cluster for running the application. 
* The master URL determines the deployment mode of the Spark application and specifies where the Spark driver should run.
* There are different type of URL that can be used:
    * local: `--master local[N]`
        * Ex: `spark-submit --master local[*] --class com.example.MyApp my-app.jar`
    * Standalone: `--master spark://<portnumber>`
        * Ex: `spark-submit --master spark://spark-master-host:7077 --class com.example.MyApp my-app.jar`
    * Apace Mesos: `--master mesos://<portnumber>`
        * Ex: `spark-submit --master mesos://mesos-master-host:5050 --class com.example.MyApp my-app.jar`
    * Hadoop YARN: `--master yarn`
        * `--deploy-mode client`: The driver runs on the machine where spark-submit is executed.
        * `--deploy-mode cluster`: The driver runs on one of the worker nodes in the cluster.
        * Ex: `spark-submit --master yarn --deploy-mode cluster --class com.example.MyApp my-app.jar`
    * Kubernetes: `--master k8s://https//<kubernetes-apiserver-host>:<kubernetes-apiserver-port>`
        * Ex: `spark-submit --master k8s://https://k8s-apiserver-host:443 --class com.example.MyApp --conf spark.kubernetes.container.image=my-spark-image my-app.jar`

#### How do you provide memory configuration and why did you uses this much memory??

* memory configuration can be provided as:
    * --conf spark.driver.memory=4g \
    * --conf spark.executor.memory=8g \
    * --conf spark.executor.cores=4 \
    * --conf spark.executor.instances = 10 # `number of executors`

* We require memory for smooth execution of application and the process should not throw out of memory error.

#### How to update configuration like broadcast join threshold, timeout, dynamic allocation??

* broadcast join threshold: `spark.sql.autoBroadcastJoinThreshold 1000` (bytes)
* broadcast timeout: `spark.sql.broadcastTimeout 600` (seconds)
* enable dynamic memory allocation: `spark.dynamicAllocation.enabled true`
* dynamic memory allocation: 
    * `spark.dynamicAllocation.minExecutors 2`
    * `spark.dynamicAllocation.maxExecutors 10`

## Deployment Mode and Edge Node in Spark

#### What is edge node??

* Edge node works as a mediator between the cluster and the user.
* Edge node is used to avoide a direct with the cluster.
* ther must be situations where if direct access given to the developer, he may manuallly copy data to the cluster node and this data cant be replicated.
* therefore, the code or data submission is done through edge node.
* also edge node profide addition security. if anyone tries to login to edge node, it can authenticate the person.

#### Whata are the deployment modes in spark

* There are 2 types of deployment modes:
    1. Client Mode:
        * if we submit code through edge node, the Driver will be created on edge node and all exectors will run of cluster.
        * Here, if we shut down the edge node, all the executors will be lost as the driver was on local machine.
        * also we will get a network latency, as the data transfer between the cluster and driver takes time.
        * Advantage: We can see all the process running on the system and does not need to rely on cluster logs.
    
    2. Cluser Mode:
        * The driver program will run on the cluster.
        * even if we shut down the edge node, the process will continue its execution. 
        * if we want to recheck the status, we get a ApplicationID through which we can access the Spark UI and check status.

#### Why do we need client and cluster mode

| Client Mode | Cluster Mode |
|---|---|
| Logs are generated on cluster machine. Its easy to debug. | Logs are generated on `stdout` or `stderr` files. its suitable for production workload. |
| Network latency is high | Network latency is low |
| High changes of Driver Out of Memory Error | Less changes of Driver Out of Memory Error |
| Driver and Executors can be lost if the Edge server is closed. | Even if edge server closed, process can continue its execution |

## Adaptive Query Execution

#### What is AQE

* It was introduce after 3.0
* AQE is the Dynamic Query Optimization technique.
* AQE optimizes the plan at runtime
* There are 4 main features of AQE:
    
1. `Dynamically coalesce shuffle partition:`
    * Assume, you have a skewed data and you have perform a `groupBy` operation.
    * Eg: Sales Dataset, with 80% sugar sales, and 20% other product sold.
    * Here, firstly the data will be shuffled, and 200 partitions will be created,
    * data with similar keys are moves to one partition.
    * There will be scenerio where multiple empty partitions will be created.
    * Here, with the static optimiation, we may have 1 task that will run for a longer time.
    * But to avode such situation,AQE comes into picture. AQE enters when there is a `shuffling` of data.
    * Here, AQE will `coalesce` the combine the partitions which has small amount of data within it.
    * it will try to create a partition that is approximately equal data compared to the skewed partion size.
    * if the coalesce partition is aprox equal to skewed partition. the process will take place.
    * Here the AQE will combine the smaller partitions, and will remmove all the empty partitions as well.
    * This will eventually reduces the number of core required to run the task. hence, save the usage of cores.
    * ![alt text](images/AQE_coalesce.png)
    * `Dynamically coalesce shuffle partition` feature will be enabled if we set `spark.sql.adaptive.enabled` and `spark.sql.adaptive.coalescePartitions.enabled` configurations are true.



2. `Dynamically Optimizing skewed Join:` (config: `spark.sql.adaptive.optimizeSkewsInRebalancePartitions.enabled`)
    * Assume, even if coalescing the different partitons,  the new partition is less then the size of partition that has the skewed data.
    * Then, AQE will use different technique.
    * here, it will check if following rules are satesfied:
        * Rule 1: if the skewed partitoin size is more then `256 MB`.
        * Rule 2: if the partition size is 5 times more then the median.
    * if both this condition satesfies, AQE will break the skewed data partition into smaller partitions, so that it will be approximately equal to the size of smaller data partition.
    * ![alt text](images/rule_based_partition.png)

3. `Dynamically Switch Join Strategy:`
    * if the tables that are going to join has size more then 10 MB, then the AQE will not update the plan.
    * But assume, if we have 2 dataframes that needed to be join df1 of size 10GB and df2 of size 1 GB.
        * Assume we wrote some fiters thats bringins the df2 size to 7 MB,
        * here we have a dataframe of size less then 10 MB,
        * while shuffling the data `local Shuffle Reader` will read the data and 
        * If the AQE is enabled, it will optimize the final logical plan and apply `Broadcast join`, insted of `sort merge join`.
        * Config to enable `local Shuffle Reader`: `spark.sql.adaptive.localShuffleReader.enabled`

4. `Converting sort-merge join to shuffled hash join:`
    * AQE converts sort-merge join to shuffled hash join when all post shuffle partitions are smaller than a threshold, 
    * the max threshold can see the config `spark.sql.adaptive.maxShuffledHashJoinLocalMapThreshold`.




#### Why do we need AQE??

### Benifits of AQE!!

* Adaptive Query Execution (AQE) in Spark provides several key benefits that enhance performance and efficiency:

    1. Dynamic Partition Pruning:
        * AQE dynamically prunes partitions that are not required for a query. 
        * This reduces the amount of data shuffled and processed, leading to significant performance improvements, especially in large datasets with many partitions.

    2. Optimized Join Strategies:
        * AQE can dynamically choose the best join strategy based on runtime statistics. 
        * For example, it can switch from a sort-merge join to a broadcast join if it detects that one side of the join is small enough to be broadcasted, which can reduce the shuffle size and improve performance.

    3. Handling Skewed Data:
        * AQE helps in mitigating data skew by dynamically coalescing and splitting shuffle partitions based on the data distribution observed at runtime. 
        * This ensures a more balanced workload across tasks, preventing performance bottlenecks caused by skewed data.

    4. Reduced Shuffle Partitions:
        * AQE can reduce the number of shuffle partitions dynamically based on the actual data size, leading to better resource utilization and reduced overhead associated with managing many small partitions.

    5. Automatic Re-optimization:
        * AQE re-optimizes query plans based on runtime statistics. This allows Spark to adjust its execution strategy if the initial assumptions made by the query optimizer do not hold true during actual execution.

    6. Improved Resource Utilization:
        * By adapting the execution plan at runtime, AQE helps in better utilization of cluster resources. 
        * It reduces unnecessary computation and I/O, leading to more efficient use of CPU, memory, and disk.

    7. Ease of Use:
        * AQE automates many optimizations that would otherwise require manual tuning and expert knowledge. 
        * This makes it easier for users to achieve optimal performance without deep expertise in Spark internals.

    8. Enhanced Query Performance:
        * Overall, the combination of these optimizations leads to significant improvements in query performance, making Spark more efficient and responsive in handling large-scale data processing tasks.

Interviewer: These benefits sound very advantageous. How does AQE compare to traditional query execution in Spark?

Candidate: Traditional query execution in Spark relies on static optimization based on the logical plan generated during query compilation. It doesn't adapt to the actual data characteristics encountered at runtime. In contrast, AQE allows Spark to make intelligent adjustments based on real-time statistics, leading to more efficient and optimized query execution. This dynamic adaptation makes AQE superior in handling diverse and unpredictable workloads, providing better performance and resource efficiency.

## cache and persisit

#### What is Cache??

* Caching is the optimization technique.
* it stores the intermediate result.
* bydefault we get `MEMORY_AND_DASK` storage level. and this cant be changed.

* Ex:

        df = spark.read.csv('path')
        df1 = some transformatinos
        df2 = df.join(df1, col1 == co2, 'inner')
        df3 =  some transformation
        df4 = df.join(df3, col1 == col2, 'inner')
        df4.show()

    * In the above case, we are using df multiple times.
    * we learned that RDD are immutable, it means always a new RDD is created.
    * here when the executor completes the operation of calculating df, then it empty its storage and takes other tasks into memory.
    * here, whenever we want to use df to join, spark recalculates the df using the DAG, which requires CPU.
    * to avoide this problem catch the `df` so that we can use the data without recalculationg.
    * cached `df` will be stored in `storage memory pool`. if `storage memory pool` get full and we are using `unified memory manager` then some partition will be stored in `executor memory pool` as well.
    * `POINT TO NOTE:` whenever the data is cached, only the partition that are fully filled are stored and partially filled partitions are discarded, this lost partition data will be recalculated.
    * to cache the data we just need to write `df.cache()`.
    * to uncache data use command `df.unpersist()`.

>> NOTE: whenever we do `show()` spark will cache only one partition and show() dosnt require all reocrds.  
>> On the other hand, if we do `count()` spark will cache all the data within it.

#### When to use Catch and Persist??

* cache is used when you have enough memory to store entire RDD.
* when you want to use the default storage level, go with Cache.
* but if you want more flexibility go with persist

#### When should we avoide caching

* When the DF is too small or easy to calculate, in such cases we shuld not use catching.

#### How to uncatch the data??

* `df.unprsist()`

#### Difference between catche and persist??

| Cache | Persist |
|---|---|
| It can only store data in `memory and disk only` | it provide more flexibility. |
| go with cahce() when you want to use the default storage caching | use persist() when you want to cache data in disk as well . |
| cache is a wrapper class built on persisit | persisit is the core class |

#### What are different storage level in Spark??

| level| Space used |CPU time |
|---|---|---|
| MEMORY_ONLY | high | low |
| MEMORY_ONLY_SER | Low  | high |
| MEMORY_AND_DISK  | high  | medium |
| MEMORY_AND_DISK_SER | low  | high |
| DISK_ONLY  | low | high |

#### Which Storage level to choose??

* its a part of experimentation.
* firstly start with memory only and then move to memory and disk followed by disk only.

## Dynamic Resouce Allocation

#### What is Dynamic Resouce Allocation?? and why dynamic resource allocation was introduced??

* In dynamic resource allocation, the resources can be reduced and increase as per requirements.
* while using the static memory allocation, the fixed number of resources were allocated to a user.
* even if the user is not using this resources, they will be reserved to the user and can't be used by other applications.
* to avoide this problem, dynamic resource allocation was introduced. 
* We need to configure our spark applicatin to use dynamic resource allocation using config: `--config spark.dynamicResourceAllocation.enable = True`
* While setting this config, we also need to specify the minimum and maximum executors to be used using congif:
    * `--config spark.dynamicAllocation.minExecutors = 5` and `--config spark.dynamicAllocation.maxExecutors = 100`
* when we set this, our application will have minimum of 5 executors but not less then this. and max upto 100 executors.
* According the usage the resources are released.
* NOTE: resource manage does not ask spark application to leave the resource if not used.
* it is done automatically.
* if the executor is idea for `60 sec` spark will release that executor and other application can use it.
* even if the executors are released, an indepedent folder named `shuffle tracking` will be available that holds all the information about the previous shuffles, so that in case the application asks for the resources again, it does not have to recalculate everything.
* `shuffle tracking` is available in every executor bydefault.
* we can chnage this default idea time of executors using configuration.
* Now if the application want the release application back, it will wait for `1s`, if it does not find its requires resources, it will ask resource manager to allocate resources.
* Spark will ask resources in `two-fold` fashion.
    * fiestly the spark will ask for `1` executor, if thats not suffecient it will double the request and ask for `2` executors and even thats not sufficient it will ask for `4` resources followed by `8` and then `16` and then `32` and so on, till the max executors reached.

#### How Resource manager provides resources in Dynamic resource allocation??

* Resource maanger does not goes to spark application to request the resourcs.
* if the executors are idea for `60sec` spark will release resource.
* if required it will ask them back in `two-fold` fashion.

#### What are the resource allocation technique we have in spark??

* There are 2 techniques:
    * Static Resource allocation
    * Dynamic resource allocation

#### What are the challenges involved with dynamic resource allocation??

* if at some point the applicatin releases its resources because of idea state and if it requires the resources back for critical execution, at that point if it does not get resources the process can fail.

#### When to avoide dynamic resource allocation??

* When you are running any critical application in PROD and its recemened not to use dnamic resouorce allocation.

#### How spark will remove or add resources??

* if the executors are idea for `60s` it will release the executor for other application.
* if application want to retain its executors, it will wait for `1s` and if it does not find the required resources, it will request executors in `two-fold` fashion.

#### Configuration needed for dynamic resource allocation??

![alt text](images/spark_config.png)

## Partition Pruning / Dynamic Partition Pruning

* ![alt text](images/code2.png)
* in the above code, if the partition prunig is not applied, the spark will read all the data and then filter the where clause.
* But when `partition pruning` is applied, the spark will firstly apply the `where` clause while reading the data and only read partition which has the specific data
* The technique in which the `where` clause is pushed down wile reading the file is called as `Predicate Pushdown`

>> `Point to NOTE:` Partition Pruning will only work when the data is partiioned on the specific key on which filter is applied.

#### Why do we need Dynamic Partition Pruning??

* ![alt text](images/Code3.png)
* In the above code we have 2 tables `orders` and `dates`.
* here we are joining 2 DF and then filtering out data on the basis of dates columns.
* In this case `partition pruning` will not work and spark will read all data, join them and then filter the data.
* TO solve this problem, `Dynamic partition pruning` was introduced. 
* Dynamic Partition Pruning was introduced in Spark 3.0 and above.
* Lest modify the above code:
* ![alt text](images/code4.png)
* The `Dynamic partition pruning` will firstly, filter out the data from `dates` table and then broadcast that filtered table onto `orders` table.
* here, the spark will read only the part files that have the above data within it.
* in this way minimum part files were read the output was desplayed.

>> Condition to apply `Dynamic Partition Pruning`
>> * Works for fast and dimension like tables. where one table and large and other table is small enough to broadcast.
>> * Fact table must be partitioned. the data should be partiton on same column on which the join is done.
>> * The small table should be broadcasted.




#### When DPP not work??

* If the large data is not paritioned on key on which it is joining with other table.
* both the tables that are joining are large and none of the table is broadcasted.
* the data is not partitioned.

## Salting

#### What is data skewness problem??

* Assume you have a dataframe will some keys and there is one particular key that is in large quantity.
* in this case, one of the parition will be large and other paritions will be small.
* this is the data skweness problem.
* here all other task will be executed, but one of the task will be very large.

#### What causes Data Skewness??

* GroupBy operation.
* skewed data from source
* join operation
* Inadequare partition strategy (bydefauly we use hash partitioning strategy)

#### Disadvantages or Cons of Data Skewness??

* Slow Running Task
* Spliting data to disk

#### What are the ways to remove skewness??

* increasing shuffle partition
* using repartition
* custom partitioning
* Broadcast table
* AQE
* Salting

#### What is salting??

* Salting is the technique through which we can remove skewness in the partitions.
* we create a new column, that will help us to devide the data partitions.


#### How can we implement salting??

## 20 Project Interview Questions

### Infrastructure Related Questions

##### What cluster manager you have used in project??

* We have worked on yarn
* we just need to mention `--master yarn` in configuration.
* inside yarn we have queue, in my organisation we had 2TB cluster of ram which was assigned to our group.
* the whole team together was using 2 TB of ram.

##### What is the size of the cluster??

* Inside cluster we had queue, which was of 2 TB

### Source and Sink related question??

##### How does your data comes into s3 bucket??

* We use a custom API that was built specifically to extract data from our various data sources. This custom API is designed to handle the specific requirements of our data extraction process, ensuring that the data is efficiently and reliably gathered.
* data was ingested into source bucket by extracting it using API
* Daily we use to get around 1TB of data from different vendors.
* other team was responsible to mange this API and get data into S3



##### What are the other sources you have used??

* we used the dimension tables, perfrom some transformation and store them into s3.

##### What is the sink in your project??

* we have s3 bucket to store data.
* we had a staging table where we write the logs of task whether it ran sucessfuly or failed.

##### What is the frequency of data in source??

* data was on daily basis.
* it was a batch data
* we use to receive data at 1 AM CST.

### In depth Question related project

##### What is the volumne that you deal with??

* we use to get a lot of data from different vendors.
* around 100 GB of data we use to collect.

##### Please explain your project in detail??

* AIM: the aim of this project was to send out a daily list of customer data to marketing team, so that they can make calls.
* we used to receive data into s3.
* we used to read data into spark. it was a parquet file format.
* we were suppose to process the data before 9 AM, and the data should be transfered before 9AM, because the daily work of marketing teaam was dependent on this list.
* we were deling with the credit card data, and it is one o the popular US based company.
* in between the process we have DQ check at the stand and end of the data processing that assure of Uniqueness, recency checks and schema check.

##### What are the challenges you have faced and how did you resolved it??

* the main challenge that i faced while creating this project was to understand the data and business logic behind it.
* there were lot of different terms that were used to represent the states of the customer.

##### Which optimization technique you have used and why??

* we used `caching`, because we used one dataframe that was used multiple time in transformation for joining with other dfs.
* also done the `broadcast` of dimension table.
* usd `AQE`
* used best practice code (i.e. not used `select * statemet`) to extract data, only important data was extracted.
* alos used `repartition` before storing data to s3.
* tried avoiding `UDFs` and tried using built in spark function
* leavraged `predicate pushdown` as early as possible to filter out data.

##### How you done spark performance tuning??

* Adjust executor memory: provided enough memory to each executor, so that the executor run effeciently without failing.
    * `--executor-memory 4G`
* Driver memory: ensured effecient memory is provided to driver for storing metadata.
    * `--driver-memory 4G`
* parallelism: set the number of partition to optimum level to ensure tasks are evenly distributed.
* Resource configuration: set the number of core and executors based on cluster configuration.
    * `--num-executors 10`
    * `--executor-cores 4`

##### can you please walk me through the spark-submit command that you have used??

    spark-submit \
    --name SparkApp \
    --master yarn \
    --deploy-mode cluster \
    --executor-memory 4G \
    --driver-memory 4G \
    --num-executors 10 \
    --executor-cores 4 \
    --conf spark.serializer=org.apache.spark.serializer.KryoSerializer \
    --conf spark.sql.shuffle.partitions=200 \
    main.py

* This is the trial and error method, where we used to change configs and test out the performance.

##### What is the size of your team??

* we had 4 developers.
* 1 tech lead
* 1 manager

##### How do you take your code to higher environment??

* Firstly, we used to Test our code in `DEV` and `UAT`.
* once everythin works fine, we used to raise a Pull request for the PROD and get it revied from the team mates.
* further deployment was done by DevOps Team.

### Scheduling related question??

##### How do you schedule your job in production??

* we used the receive file on daily basis.
* the files were lading in S3 in between 1 AM - 2 PM.
* EDD functionality were buidl to trigger dag, once the faile was landed in bucket.
* it was taking around 1 Hr to run the entite pipeline.
* the file was delivered at around 3 - 4 AM.

##### What is the frequency of your job??

* daily basis, the job was used to start at around 2 AM.

##### How much time it takes to run a job??

* Daily we used to get 200 GB of data, it use to take 1 hr to 1.30 hr to run the DAG.

##### How do you notify business users if your job fails??

* there is a Pagerduty softwarwe that we used to monitor our pipelines.
* if the dag didnt start as expected time, we used to get the alert
* also if any task fails we use to get a failure ticket for it on pagerduty.


##### How do you reprocess the data if job fails??

* There are mainy resons for a job failure.
* one common reason was that the session was killed at run time.
* in this situation we to recreate cluster and rerun the dag.

* if we missed to process a data from perticular day.
* we used to catchup mode, that will pick the dated file we want to process.

### Business Related Question

##### Who is your client??

* i cant revil my client info.

##### What is the impact of the project on the business??

* the data was reached to market team for further calling on time.
* which eventually helped in gaining more customer and increase revenue.

##### What is the data domain??

* it was a credit card data.

##### How long did it took to make this project

* it took us around 4 months to complete the project.